## 🎵 KaraFan

The **BEST music separation** software with help of A.I. ... **to my ears** ! 👂👂

#### 📝 The Story of this project, Explanations, and Thanks are on my  &nbsp;[![GitHub](https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&labelColor=333&logoColor=white&color=ff0080)](https://github.com/Captain-FLAM/KaraFan)  &nbsp;page.

**R.T.F.RM** --> Read [The Fucking READ ME](https://github.com/Captain-FLAM/KaraFan#readme) - (GitHub home page), it's worth it ! 😁  
**R.T.F.W** &nbsp;&nbsp;--> Read [The Fucking WIKI](https://github.com/Captain-FLAM/KaraFan/wiki) (**updated continuously**)

### 💲 And if you like this software :

[![ko-fi](images/ko-fi.png)](https://ko-fi.com/captain_flam)

### 📜 LICENSE - &copy; Captain FLAM & Jarredou - 2023 - M.I.T license

That means if you want to use any part of our code in your projects, **YOU HAVE TO let the copyrights in place !!**  
Copyright is not a shame, it's a way to say thanks to the authors for their work !

### ⏹️ If you want to stop the processing **use : <kbd>CTRL</kbd> + <kbd>C</kbd>**

In [ ]:
import os, sys, subprocess

# !!!!  If you are a developer  !!!!
#
# If you want to work on KaraFan on your PC or on your Google Drive (with Google Colab),
# this option Deactivate the "Automagic Updates", and so don't overwrite your modifications.
# Moreover, on Colab, it will install the code in "/KaraFan" on your Google Drive. 

I_AM_A_DEVELOPER = False

# If you still have GPU memory errors, you can try to
# uncomment the following line, but I'm not sure it will work...
# Anyway, following the Nvidia documentation,
# they recommend to set this environment variable to 512 MB :

# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:512"

#****************************************************************

Repo_url = "https://github.com/Captain-FLAM/KaraFan"

Gdrive = ""; Project = ""; User = ""; isColab = False
try:
	# Are we on Google Colab ?
	from google.colab import drive
	isColab = True
	drive.mount("/content/Gdrive", timeout_ms=60000)  # 60 seconds
	Gdrive  = "/content/Gdrive/MyDrive"
	Project = os.path.join(Gdrive if I_AM_A_DEVELOPER else "/content", "KaraFan")
except:
	if not isColab:
		# We are on PC
		Project = os.getcwd()  # Get the current path
		Gdrive  = os.path.dirname(Project)  # Get parent directory

if Gdrive != "":
	if not os.path.exists(Project):
		print('Cloning "KaraFan" Repository...')
		os.chdir(os.path.dirname(Project))
		try:
			subprocess.run(["git", "clone", f"{Repo_url}.git", "KaraFan"], text=True, capture_output=True, check=True)
		except subprocess.CalledProcessError as e:
			print("Error during Cloning :\n" + e.stderr + "\n" + e.stdout + "\n")
			exit(1)

	if os.path.exists(Project):
		# Mandatory to import App modules
		sys.path.insert(0, Project)
		import App.setup

		params = {"Gdrive": Gdrive, "Project": Project, "isColab": isColab, "I_AM_A_DEVELOPER": I_AM_A_DEVELOPER}
	else:
		print('"KaraFan" is NOT installed !');  exit(1)

## 😎 This is it !

In [ ]:
if Gdrive != "": App.setup.Install(params); import App.main; App.main.Start(params)
else: print("\n****  You have denied access to your Drive !  ****\n") 